## Script para análise de genes 
##### Este script foi desenvolvido com o intuito de analisar 2 genes resistentes a antibióticos , MexA e FosA, da Pseudomonas aeruginosa.
##### É possível fazer uma procura no Pubmed, NCBI e Uniprot e também fazer um Blast das sequências dos genes.

In [1]:
from Bio import Entrez, SeqIO,SearchIO
from Bio.Blast import NCBIWWW, NCBIXML
from Bio import SwissProt
from Bio import ExPASy
from Bio import AlignIO
from Bio import Phylo
import re
Entrez.email = "grupo2labsbioinf@gmail.com"

#### Análise da literatura com base no Pubmed

In [2]:
def pubmed(termo):
    '''
    A função pubmed depende do package Biopython Entrez para fazer uma pesquisa
    no site pubmed a cerca de artigos relevantes para o estudo dos genes escolhidos.
    Para a função é dado o termo, ou seja, o que queremos pesquisar no site e é 
    retomado um conjunto de artigos relevantes sobre o assunto.
    '''
    handle = Entrez.esearch(db="pubmed", sort='relevance', term=termo)
    record = Entrez.read(handle)
    lista = record["IdList"]
    pubmed_ft=open("pubmed_ft.txt","w",encoding="utf8")
    print ("\n ========= Artigos da Pubmed ========= \n")
    for index in range(0, len(lista)):
        listId = lista[index]
        handle = Entrez.esummary(db="pubmed", id=listId)
        record = Entrez.read(handle)
        print (index+1)
        pubmed_ft.write(str(index+1)+":\n")
        print ("Título: "+record[0]["Title"])
        pubmed_ft.write(("Título: "+record[0]["Title"])+"\n")
        print ("Autores: "+", ".join(record[0]["AuthorList"]))
        pubmed_ft.write("Autores: "+", ".join(record[0]["AuthorList"])+"\n")
    pubmed_ft.close()

#### Análise das informações dos genes, nomeadamente anotações, features e qualifiers.

In [3]:
def genefiles(filename,start,stop): 
    '''
    A função genefiles depende do package Biopython Entrez para guardar ficheiros
    genbank dos genes a estudar, a partir do NCBI.
    Para a função é dado o nome do ficheiro a criar e a localização do codão start 
    e stop da sequência e é retomado o ficheiro genbank.
    '''
    fetch_handle = Entrez.efetch(db = "nucleotide", rettype = "gbwithparts",retmode = "text", id = "NC_002516.2", seq_start = start,seq_stop = stop)
    dna_fetch_record = fetch_handle.read()
    with open(filename, "w") as out_handle: 
        out_handle.write(dna_fetch_record)
    return fetch_handle


In [4]:
def geneinfo(filename):
    '''
    Para a função geneinfo é dado o nome do ficheiro genbank do gene a procurar
    informações e é imprimido várias informações acerca dele como o nome, acession
    number,...Imprime também informações específicas para recursos "CDS" no ficheiro 
    genbank
    '''
    flag = 0
    print ("\n ========= Informações acerca do gene ========= \n")
    with open(filename) as file: 
        file_genes = list(SeqIO.parse(file, "gb"))
    for i in range(len(file_genes)):
        print("Nome:",file_genes[i].name)
        print("Descrição:",file_genes[i].description)
        print("Acession Number:",file_genes[i].id)
        print("Tamanho da sequência:",len(file_genes[i].seq),"bp")
        print("DBxrefs:",file_genes[i].dbxrefs)
        print ("Source:",file_genes[i].annotations["source"])
        print ("Organismo:",file_genes[i].annotations["organism"]) 
        print ("Taxonomia:",file_genes[i].annotations["taxonomy"])
        print ("Topologia:",file_genes[i].annotations["topology"])
        print ("Tipo de molécula:",file_genes[i].annotations["molecule_type"])
        print ("Acessions:",file_genes[i].annotations["accessions"])
        for feat in file_genes[i].features: 
            if feat.type == "CDS" and flag == 0: 
                flag += 1
                print (f"\nFeature type = CDS")
                print("ID da proteína:",feat.qualifiers["protein_id"])
                print("Produto:",feat.qualifiers["product"])
                print("Tradução:",feat.qualifiers["translation"])
                print("Note:",feat.qualifiers["note"])
                print("X-refs:",feat.qualifiers["db_xref"],end="\n\n")
        return None

#### Blast- Análise de homologias entre sequências

In [5]:
def blast_files(programa,file_gb,file_fasta,id_seq,blast):
    '''
    A função blast_files converte ficheiros genbank a fasta para depois serem lidos
    e ser feito um blast sobre a sequência fasta.
    Para a função é dado o nome do ficheiro genbank existente, o nome do ficheiro
    fasta a criar e o nome do ficheiro xml a criar com o conteúdo obtido pelo blast.
    Nesta função é retomado o ficheiro xml com o blast.
    '''
    if programa == "blastn":
        count=SeqIO.convert(file_gb, "genbank", file_fasta, "fasta")
        record_1 = SeqIO.read(open(file_fasta), format="fasta")
        result_handle = NCBIWWW.qblast(programa,"nt", record_1.format("fasta"), entrez_query="NOT pseudomonas aeruginosa[organism]")
    else:
        result_handle = NCBIWWW.qblast(programa, "nr",id_seq, entrez_query="NOT pseudomonas aeruginosa[organism]")
    save_file = open(blast, "w")
    save_file.write(result_handle.read())
    save_file.close()
    result_handle.close()
    result_handle = open(blast)
    blast_records = NCBIXML.parse(result_handle)
    return result_handle

In [6]:
def blastinfo(filename, e_value):
    '''
    Para a função blastinfo é dado o nome do ficheiro xml criado com os resultados
    do blast e imprime alguns parâmetros usados, o alinhamento máximo, o comprimento
    e a pontuação máxima, e os seus acession numbers dos resultados do blast.
    '''
    print ("\n ========= Informações acerca do Blast ========= \n")
    with open(filename) as file:
        blast_record = NCBIXML.read(file)
    E_VALUE_THRESH = e_value
    count = 0
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            if hsp.expect < E_VALUE_THRESH:
                count += 1
                print("****Alinhamentos****")
                print("Sequência:", alignment.title)
                print("Tamanho:", alignment.length)
                print ("E value:", hsp.expect)
                print(hsp.query[0:75] + "...")
                print(hsp.match[0:75] + "...")
                print(hsp.sbjct[0:75] + "...")
                print()           
    print(f"Existem {count} sequências similares no output do Blast.")      
    return None

#### Análise das proteínas através da Swissprot

In [7]:
def swissprot_files(id_prot,filename):
    '''
    A função swissprot_files utiliza o package Biopython ExPASy para ir buscar ficheiros à Uniprot.
    Para a função é dado o id do gene a procurar na base de dados e o nome do ficheiro xml a criar 
    com os resultados obtidos. 
    '''
    handle = ExPASy.get_sprot_raw(id_prot)
    gene = handle.read()
    handle.close ()
    with open(filename, "w") as out_handle:
        out_handle.write(gene)
    return gene

In [8]:
def swissprot(filename):
    '''
    Para a função swissprot é dado o nome do ficheiro html obtido com os resultados da swissprot e 
    imprime o ID com o status reviwed e o comprimento da sequência, o nome do gene, source, mutações, 
    funções,...
    '''
    print ("\n ========= Informações acerca da proteína ========= \n")
    with open(filename) as file: 
        file_list = file.readlines()
    for i in range(len(file_list)):
        if "ID" in file_list[i][:5]: 
            print("Nome de acesso, Status e comprimento da sequência:",file_list[i][5:].replace("    ",""),end="")
        if "AC   " in file_list[i][:5]: 
            print("ID:",file_list[i][5:],end="")
        if "DE   " in file_list[i][:5]: 
            print(file_list[i][5:],end="")
        if "GN" in file_list[i][:5]: 
            print(file_list[i][5:],end="")
        if "OS" in file_list[i][:5]: 
            print("Source:",file_list[i][5:],end="")
        if "SUBCELLULAR LOCATION:" in file_list[i][:30]: 
            print("Localização subcelular:",file_list[i][30:],end="")
        if "DR" in file_list[i][:5] and "PROSITE" in file_list[i][:15]:
            print("Domínios PROSITE:",file_list[i][5:],end="")
        if "RP" in file_list [i][:5] and 'FUNCTION' in file_list [i]:
            print(file_list[i][5:], end="")
        if "CC   "  in file_list [i][:5] and not "CC   --" in file_list [i] and not  "CC   Co" in file_list [i] and not  "CC   Di" in file_list [i]: 
            print (file_list [i][5:], end="")
    f = open(filename, "r")
    content = f.read()
    if "FT   MUTAGEN         " in content:
        ex_reg = r"MUTAGEN[^/]*/[^/]*/"
        match = re.findall(ex_reg,content)
        match1 = re.search(r"/[^/]*", match[1])
        teste = re.sub(r"FT"," ",match1.group())
        print ("Mutação:",teste)
    return None

#### Alinhamento Múltiplo

In [9]:
def alinhamento_multiplo(filename):
    '''A função alinhamento_multiplo recebe como parâmetro um ficheiro fasta que contém os alinhamentos múltiplos
    provenientes da utilização do docker para o acesso ao ClustalW e respetiva manipulação dos alinhamentos. 
    '''
    print ("\n ========= Informações acerca do alinhamento múltiplo ========= \n")
    alinhamento=AlignIO.parse(filename,"fasta")
    for align in alinhamento:
        print(align)
        print ("Tamanho do alinhamento %i" % align.get_alignment_length())
    return alinhamento

#### Árvore filogenética

In [10]:
def arvore(filename):
    '''A função arvore recebe como parâmetro um ficheiro no formato newick proveniente da utilização do docker 
    para o acesso ao ClustalW.
    '''
    print ("\n ========= Informações acerca da filogenia ========= \n")
    tree = Phylo.read(filename, "newick")
    Phylo.draw_ascii(tree)
    return None

#### Menu de escolha das funções 

In [11]:
def menu():
    '''Função menu onde é possível escolher a função de interesse realtivamente ao que se quer visualizar'''
    while True:
        print("""
        =========== Funções ===========
        0.Sair
        1.Procurar na Pubmed
        2.Criar ficheiro gb do gene
        3.Procurar informação do gene
        4.Criar ficheiro xml do Blast
        5.Procurar informação sobre o Blast
        6.Criar ficheiro xml da Swissprot
        7.Procurar informação da Swissprot
        8.Alinhamento Múltiplo
        9.Árvore Filogenética
        """)
        op = input("Escolha uma opção:")
        if op == "1": 
            termo=input("Indique o que quer pesquisar: ")
            pubmed(termo)
        
        elif op == "2": 
            n = int(input ("Escolha o gene: MexA(1) ou FosA (2)"))
            if n == 1:
                start = 472024 
                stop = 473175
                filename = "mexA.gb"
                genefiles (filename, start,stop)
            elif n == 2:
                start = 1221691
                stop = 1222098
                filename = "fosA.gb"
                genefiles (filename, start,stop)
            else:
                print ("Opção inválida!")
                           
        elif op == "3": 
            n = int(input("Escolha o gene: MexA(1) ou FosA (2)"))
            print ()
            if n == 1: 
                geneinfo("mexA.gb")
            elif n == 2: 
                geneinfo("fosA.gb")
            else: 
                print ("Opção inválida!")
                           
        elif op == "4": 
            n = int(input ("Escolha o gene: MexA(1) ou FosA (2)"))
            blast = int(input("Escolha o programa de Blast: blastn(1) ou blastp (2)"))
            if n == 1:
                if blast == 1:
                    blast_files ("blastn","mexA.gb","mexA.fasta",None,"Blast_mexA.xml")
                else:
                    blast_files ("blastp",None,None,"P52477","Blastp_mexA.xml")
            elif n == 2:
                if blast == 1:
                    blast_files("blastn","fosA.gb","fosA.fasta",None,"Blast_fosA.xml")
                else:
                    blast_files("blastp",None,None,"Q9I4K6","Blastp_fosA.xml")
            else: 
                print ("Opção inválida!")
                           
        elif op == "5": 
            n = int(input("Escolha o gene: MexA(1) ou FosA (2)"))
            blast = int(input("Escolha o programa de Blast: blastn(1) ou blastp (2)"))
            e_value = float (input("Introduza um valor para o E-value do Blast:"))
            if n == 1: 
                if blast == 1:
                    print("Acession numbers in file:",blastinfo("Blast_mexA.xml",e_value))
                else:
                     print("Acession numbers in file:",blastinfo("Blastp_mexA.xml",e_value))
            elif n == 2: 
                if blast == 1:
                    print("Acession numbers in file:", blastinfo("Blast_fosA.xml",e_value))
                else:
                    print("Acession numbers in file:",blastinfo("Blastp_fosA.xml",e_value))
            else: 
                print ("Opção inválida!")
                                    
        elif op == "6": 
            n = int(input("Escolha o gene: MexA(1) ou FosA (2)"))
            if n == 1: 
                id_prot = "P52477"
                filename = "prot_mexA.xml"
                swissprot_files (id_prot,filename)
            elif n == 2: 
                id_prot = "Q9I4K6"
                filename = "prot_fosA.xml"
                swissprot_files (id_prot,filename)
            else: 
                print ("Opção inválida!")
                           
        elif op == "7": 
            n = int(input("Escolha o gene: MexA(1) ou FosA (2)"))
            if n == 1: 
                swissprot ("prot_mexA.xml")
                filename="prot_mexA.xml"
            elif n == 2: 
                swissprot ("prot_fosA.xml")
                filename="prot_fosA.xml"
            else: 
                print ("Opção inválida!")
                
        elif op == '8':
            n = int(input("Escolha o gene: MexA(1) ou FosA (2)"))
            if n == 1: 
                alinhamento_multiplo ("clustal_mexa.fas")
                filename="clustal_mexa.fas"
            elif n == 2:
                alinhamento_multiplo ("clustal_fosa.fas")
                filename="clustal_fosa.fas"
            else: 
                print ("Opção inválida!") 
                
        elif op == '9':
            n = int(input("Escolha o gene: MexA(1) ou FosA (2)"))
            if n == 1: 
                arvore ("tree_mexa.nwk")
                filename="tree_mexa.nwk"
            elif n == 2:
                arvore ("tree_fosa.nwk")
                filename="tree_fosa.nwk"
            else: 
                print ("Opção inválida!") 
                           
        elif op == "0":
            break
        else:
            print("\n Opção inválida!")
    print("\n Adeus")


if __name__ == "__main__":
    menu()
    pass


        =========== Funções ===========
        0.Sair
        1.Procurar na Pubmed
        2.Criar ficheiro gb do gene
        3.Procurar informação do gene
        4.Criar ficheiro xml do Blast
        5.Procurar informação sobre o Blast
        6.Criar ficheiro xml da Swissprot
        7.Procurar informação da Swissprot
        8.Alinhamento Múltiplo
        9.Árvore Filogenética
        


Escolha uma opção: 1
Indique o que quer pesquisar:  mexA pseudomonas aeruginosa



 ========= Artigos da Pubmed ========= 

1
Título: Molecular detection of gyrA and mexA genes in Pseudomonas aeruginosa.
Autores: Abed WH, Kareem SM
2
Título: Effect of single-dose carbapenem exposure on transcriptional expression of bla<sub>NDM-1</sub> and mexA in Pseudomonas aeruginosa.
Autores: Choudhury D, Paul D, Ghosh AS, Das Talukdar A, Dutta Choudhury M, Maurya AP, Dhar Chanda D, Chakravarty A, Bhattacharjee A
3
Título: Evaluation of efflux pump activity and biofilm formation in multidrug resistant clinical isolates of Pseudomonas aeruginosa isolated from a Federal Medical Center in Nigeria.
Autores: Ugwuanyi FC, Ajayi A, Ojo DA, Adeleye AI, Smith SI
4
Título: Changes in the resistance and epidemiological characteristics of Pseudomonas aeruginosa during a ten-year period.
Autores: Feng W, Huang Q, Wang Y, Yuan Q, Li X, Xia P, Sun F
5
Título: MexAB-OprM Efflux Pump Interaction with the Peptidoglycan of <i>Escherichia coli</i> and <i>Pseudomonas aeruginosa</i>.
Autores: Ma M, Lu

Escolha uma opção: 1
Indique o que quer pesquisar:  fosA pseudomonas aeruginosa



 ========= Artigos da Pubmed ========= 

1
Título: Development of antibiotic resistance in the ocular Pseudomonas aeruginosa clone ST308 over twenty years.
Autores: Khan M, Willcox MDP, Rice SA, Sharma S, Stapleton F
2
Título: Draft genome sequence of KPC-2-producing Pseudomonas aeruginosa recovered from a bloodstream infection sample in Brazil.
Autores: de Oliveira Santos IC, Albano RM, Asensi MD, D'Alincourt Carvalho-Assef AP
3
Título: SPM-1-producing Pseudomonas aeruginosa ST277 carries a chromosomal pack of acquired resistance genes: An example of high-risk clone associated with 'intrinsic resistome'.
Autores: Galetti R, Andrade LN, Varani AM, Darini ALC
4
Título: Whole-genome sequence of multi-drug resistant Pseudomonas aeruginosa strains UY1PSABAL and UY1PSABAL2 isolated from human broncho-alveolar lavage, Yaoundé, Cameroon.
Autores: Madaha EL, Mienie C, Gonsu HK, Bughe RN, Fonkoua MC, Mbacham WF, Alayande KA, Bezuidenhout CC, Ateba CN
5
Título: Inhibition of Fosfomycin Resistan

...


 ========= Informações acerca da filogenia ========= 

                                                , TGC25793.1_11-388
  ______________________________________________|
 |                                              | WP_143096612.1_1-378
 |
 |_________________________________________ WP_043248663.1_1-382
_|
 |                                                   , WP_049317199.1_1-383
 |                                                   |
 |                                                   | _ WP_049324879.1_1-383
 |___________________________________________________||
                                                     || SVJ58457.1_1-383
                                                     ||
                                                     |, SSU20340.1_60-442
                                                      |
                                                      | WP_003118819.1_1-383
                                                      |
                           

Escolha uma opção: 9
Escolha o gene: MexA(1) ou FosA (2) 2



 ========= Informações acerca da filogenia ========= 

  _____________ WP_099758231.1_1-136
 |
 |___________ WP_095138516.1_1-136
 |
_|                                                   , SSU20817.1_1-109
 |                                                   |
 |                                                   | WP_003158225.1_1-135
 |                                                   |
 |___________________________________________________|, AAT49669.1_1-135
                                                     ||
                                                     || WP_023107004.1_1-135
                                                     ||
                                                     ||_ WP_003163118.1_1-135
                                                      |
                                                      | WP_033976670.1_1-135
                                                      |
                                                      , sp|Q9I4K6|FOSA_PSEAE
  

Escolha uma opção: 0



 Adeus
